In [1]:
import sqlite3
import pandas as pd

andmebaasi_nimi = "v33_koondkorpus_sentences_verb_pattern_obl_20241002-130310.db"
tabel = "transaction_row"

In [2]:
conn = sqlite3.connect(andmebaasi_nimi)

# Valime välja kõik võimalikud erinevad 'feats' väljad kui deprel='obl'
# selleks, et alles jätta ainult 'feats' väljad, mis sisaldavad kohakäänet
query = f"""
SELECT DISTINCT feats
FROM {tabel}
WHERE deprel = 'obl'
"""

result_uus = pd.read_sql_query(query, conn)
conn.close()

In [3]:
result_uus = result_uus.assign(split_feats=lambda df:df['feats'].str.split(',')) # jagame 'feats' lahtri mitmeks
# uus veerg, mis näitab lemma kohakäänet
result_uus['case']='-'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'abl' in x),'case'] = 'abl'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'ad' in x),'case'] = 'ad'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'all' in x),'case'] = 'all'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'adt' in x),'case'] = 'adt'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'el' in x),'case'] = 'el'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'ill' in x),'case'] = 'ill'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'in' in x),'case'] = 'in'
result_uus = result_uus[result_uus['case'] != '-'] # eemaldame read, kus ei ole kohakäänet

# uus veerg, mis näitab, kas lemma on 'sg' või 'pl' (ainsus või mitmus)
result_uus['number']='-'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'sg' in x),'number'] = 'sg'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'pl' in x),'number'] = 'pl'

# uus veerg, mis näitab, kas lemma on proper (1) või mitte (0) 
result_uus['proper']='0'
result_uus.loc[result_uus['split_feats'].map(lambda x : 'prop' in x),'proper'] = '1'

result_uus = result_uus.drop(columns=['split_feats']) # seda lahtrit pole enam vaja

In [4]:
result_uus # loodud dataframe

,feats,case,number,proper
0,"com,in,sg",in,sg,0
2,"all,com,pl",all,pl,0
4,"ad,sg",ad,sg,0
5,"com,el,pl",el,pl,0
6,"ad,com,sg",ad,sg,0
...,...,...,...,...
366,"ill,nominal,pl",ill,pl,0
370,"in,pl,super",in,pl,0
371,"in,mod,ps,sup",in,-,0
399,"in,main,ps,sup",in,-,0


In [5]:
df_feats_set = set(result_uus['feats'])
feats_list = tuple(df_feats_set)

conn = sqlite3.connect(andmebaasi_nimi)

# valime andmebaasist kõik read, kus leidub kohakääne (result_uus 'feats' põhjal) ning deprel = obl
query = f"""
SELECT *
FROM {tabel}
WHERE feats IN {feats_list} AND deprel = 'obl'
"""
    
db_df = pd.read_sql(query, conn)

# ühendame andmebaasi ja result_uus dataframe'i 'feats' veeru põhjal  
merged_df = pd.merge(db_df, result_uus, on='feats', how='left') 
conn.close()

merged_df # dataframe, kus on veerud number, case ja proper ning ainult kohakäändega read, kus deprel = obl

,id,head_id,loc,loc_rel,deprel,form,lemma,feats,parent_loc,pos,case,number,proper
0,1,2,3,-1,obl,lõpus,lõpp,"com,in,sg",None,S,in,sg,0
1,7,3,12,1,obl,keeltele,keel,"all,com,pl",None,S,all,pl,0
2,19,10,4,1,obl,sul,sina,"ad,sg",None,P,ad,sg,0
3,22,11,3,1,obl,tundidest,tund,"com,el,pl",None,S,el,pl,0
4,23,11,6,2,obl,juhul,juht,"ad,com,sg",None,S,ad,sg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7416266,54050454,30078971,7,1,obl,mulle,mina,"all,sg",None,P,all,sg,0
7416267,54050465,30078974,5,1,obl,mulle,mina,"all,sg",None,P,all,sg,0
7416268,54050472,30078981,31,-1,obl,sulle,sina,"all,sg",None,P,all,sg,0
7416269,54050473,30078981,33,1,obl,tegelt,tege,"abl,com,sg",None,S,abl,sg,0


In [6]:
# salvestame merged_df andmebaasina
merged_df = merged_df.where(pd.notna(merged_df), None) # Teeme NaN -> None (vajalik, et ei tuleks probleemi andmetüüpidega)

conn = sqlite3.connect(andmebaasi_nimi)

merged_df.to_sql('kohakaanded', conn, if_exists='replace', index=False) # uus tabel 'kohakaanded' andmebaasis

conn.close()